In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import functions    # my own functions which are used in more notebooks
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
from itertools import chain
from sklearn.metrics.pairwise import cosine_similarity
import gensim
import operator
from gensim.models import Word2Vec
import gensim.downloader as api
import ast
import re
from gensim import similarities,corpora,models

## Sources

- **List of articles with information and predicted score** -> final_articles_score_table_w2v.csv
- **Feauture importance of words** - score_of_word_with_info.csv
- **Trained W2V model** - model_w2v.model

In [3]:
score_df = pd.read_csv("outputs/classifier/train_2021/final_articles_score_table_w2v.csv")
score_of_word_with_info = pd.read_csv("outputs/classifier/train_2021/score_of_word_with_info.csv")
model_w2v = gensim.models.Word2Vec.load("outputs/w2v/w2v_published_between_"+str(2019) + " and "+ str(2022)+".model")

In [4]:
score_df

,Unnamed: 0,score,abstract,abstract_cleaned,doi,Year,authors,journal,len,abstract_cleaned_tok,top_n_words
0,1,0.868661,Neuropsychiatric symptoms caused by COVID-19,neuropsychiatric symptom caused covid-19,10.31234/osf.io/38c7u,2021.0,"Inasaridze, Ketevan",NaN,40,"['neuropsychiatric', 'symptom', 'caused', 'cov...","{'symptom': 0.95, 'neuropsychiatric': 0.93, 'c..."
1,4,0.855088,Whether monoclonal antibodies are able to neut...,whether monoclonal antibody able neutralise sa...,10.1093/infdis/jiab355,2021.0,"Widera, Marek; Wilhelm, Alexander; Hoehl, Seba...",J Infect Dis,294,"['whether', 'monoclonal', 'antibody', 'able', ...","{'imdevimab': 1.0, 'casirivimab': 1.0, 'pseudo..."
2,7,0.847378,South Korea is experiencing the community tran...,south korea experiencing community transmissio...,10.1101/2021.12.25.21268301,2021.0,"Kim, D.; Jo, J.; Lim, J.-S.; Ryu, S.",NaN,197,"['south', 'korea', 'experiencing', 'community'...","{'omicron': 1.0, 'b11529': 0.99, 'sars-cov-2':..."
3,8,0.846596,"In this study, we analyzed daily records of ne...",study analyzed daily record newly diagnosed ca...,10.1142/s0219477521500528,2021.0,"Wang, Jian Shao Wei Yan Yan Kim Junseok",Fluctuation and Noise Letters,511,"['study', 'analyzed', 'daily', 'record', 'newl...","{'wuhan': 1.0, 'multifractal': 1.0, 'hubei': 1..."
4,11,0.844665,The Covid-19 pandemic is still raging [...].,covid-19 pandemic still raging,10.3390/pediatric13010013,2021.0,"Caselli, Désirée; Aricò, Maurizio",Pediatr Rep,30,"['covid-19', 'pandemic', 'still', 'raging']","{'raging': 0.99, 'covid-19': 0.87, 'pandemic':..."
...,...,...,...,...,...,...,...,...,...,...,...
281663,281747,0.020017,BACKGROUND There is surprisingly little compar...,surprisingly little comparative published data...,10.1093/jac/dkac026,2022.0,"Noel, Alan R; Attwood, Marie; Bowker, Karen E;...",The Journal of antimicrobial chemotherapy,1293,"['surprisingly', 'little', 'comparative', 'pub...","{'finding': 0.94, 'systematic': 0.78, 'data': ..."
281664,281748,0.019655,Video 1Multiple sewing needles are retrieved f...,video sewing needle retrieved stomach endoscop...,10.1016/j.vgie.2021.01.007,2021.0,"Morita, Shinichi; Yagi, Kazuyoshi; Hoshi, Taka...",VideoGIE : an official video journal of the Am...,74,"['video', 'sewing', 'needle', 'retrieved', 'st...","{'retrieved': 0.62, 'using': 0.16, 'sewing': 0..."
281665,281749,0.017440,"The pedicled anterolateral thigh flap, with or...",pedicled anterolateral thigh flap without vast...,10.1097/gox.0000000000003774,2021.0,"Prasidha, Ines; Moisidis, Elias; Hsieh, Frank",Plast Reconstr Surg Glob Open,315,"['pedicled', 'anterolateral', 'thigh', 'flap',...","{'deep': 0.55, 'vessel': 0.35, 'epigastric': 0..."
281666,281750,0.014544,The aim of this study was to examine the acute...,aim study examine acute effect football soccer...,10.3390/ijerph182211941,2021.0,"Fernández-Ruiz, Vicente; López-Samanes, Álvaro...",Int J Environ Res Public Health,1003,"['aim', 'study', 'examine', 'acute', 'effect',...","{'−013': 0.61, '0833': 0.59, '±': 0.58, 'acute..."


In [5]:
score_of_word_with_info

,Unnamed: 0,word,score,cnt_in_all_articles,cnt_of_articles,index,first_year,quantile,doi,cnt_of_words
0,0,mdd,0.999959,1001,284,mdd,1990.0,10,['10.1007/s40273-021-01019-4' '10.1016/j.jad.2...,"[16, 12, 12]"
1,1,ocd,0.999880,887,204,ocd,2006.0,10,['10.2196/26715' '10.3389/fpsyt.2021.677567' '...,"[15, 14, 12]"
2,2,delirium,0.999785,2667,787,delirium,2001.0,10,['10.1186/s12912-021-00543-0' '10.1186/s13063-...,"[18, 17, 17]"
3,3,rhinitis,0.999598,870,482,rhinitis,1977.0,10,['10.2196/33941' '10.3390/jcm10143183' '10.117...,"[8, 8, 7]"
4,4,ibs,0.999479,475,113,ibs,2003.0,10,['10.53350/pjmhs211572062' '10.1111/jgh.15466'...,"[14, 14, 14]"
...,...,...,...,...,...,...,...,...,...,...
4995,4995,cobalamin,0.684859,67,26,cobalamin,1996.0,1,['10.3389/fmed.2021.807017' '10.3390/nu1306191...,"[3, 2, 2]"
4996,4996,sgm,0.684791,201,46,sgm,2013.0,1,['10.1002/jia2.25728' '10.2147/dddt.s288829' '...,"[14, 10, 9]"
4997,4997,casp,0.684779,96,82,casp,2016.0,1,['10.3390/bioengineering9030118' '10.1002/prot...,"[3, 3, 3]"
4998,4998,methodswe,0.684670,129,129,methodswe,2005.0,1,['10.1101/2020.12.30.20248929' '10.1101/2021.0...,"[1, 1, 1]"


#### Calculate word vectors:

In [6]:
words = model_w2v.wv.key_to_index.keys()
we_dict = {word:model_w2v.wv[word] for word in words}
words_list_total = pd.DataFrame(we_dict.items())
words_list_total.columns = ["word","vector"]
word_vectors = pd.DataFrame(words_list_total["vector"].tolist(),index=words_list_total["word"])
word_vectors = word_vectors.transpose()

## Entities filtering

**Columns**
- word -> resulted filtered words from corpus 
- score -> feature importance of word by SMRE (LR-W2V)
- cnt_of_articles -> cnt of unique articles where word is (whole CORD-19 corpus)
- cnt_in_all_articles -> total number of words in corpus (whole CORD-19 corpus)
- First_year -> The oldest article and its publication year where the word appeared.
- dois_by_score_of_article -> articles where is the word and articles with the highest value of predicted score.
- dois_by_cnt_in_article -> articles with the highest number of words appeared in.
- top_sim_words -> the most similary words with cosine distance to these word

New columns
- quantile -> quantile of feature importance of word. If quantile value = 10 -> FI is highest.
- sim_to_xxx -> distance of word to selected word (representation of wanted domain - there drug and remdesivir).
- customized_score_with_similar -> score (fi) multiplied by cnt_of_articles and sim_to_xxx
- customized_score -> score (fi) multiplied by cnt_of_articles

In [7]:
score_of_word_with_info_f = score_of_word_with_info.copy()

In [8]:
# add n top similar words
score_of_word_with_info_f = functions.add_similar_words(score_of_word_with_info_f,score_df,model_w2v)
score_of_word_with_info_f["customized_score"] = score_of_word_with_info_f["score"]*score_of_word_with_info_f["cnt_of_articles"]
score_of_word_with_info_f["quantile"]=pd.cut(score_of_word_with_info_f.score, bins=10, right=True,labels=["1","2","3","4","5","6","7","8","9","10"])

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:30<00:00, 164.98it/s]


In [9]:
selected_domain_list = [["omicron"], ["covaxin"], ["b11529"],["remdesivir"],["imdevimab"]]
#selected_domain_list = [["remdesivir"]]
for domain in selected_domain_list:
    append_sim = []
    for word in tqdm(list(score_of_word_with_info_f.word.values)):
        append_sim.append(model_w2v.wv.n_similarity(domain,[word])) 
    score_of_word_with_info_f["sim_to_"+str(domain)] = append_sim
    score_of_word_with_info_f["customized_score_with_similar_"+str(domain)] = score_of_word_with_info_f["score"]*score_of_word_with_info_f["cnt_of_articles"]*score_of_word_with_info_f["sim_to_"+str(domain)]

100%|███████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 17525.70it/s]


In [11]:
score_of_word_with_info_f.sort_values("sim_to_['remdesivir']",ascending=False)[:10]

,Unnamed: 0,word,score,cnt_in_all_articles,cnt_of_articles,index,first_year,quantile,doi,cnt_of_words,...,sim_to_['omicron'],customized_score_with_similar_['omicron'],sim_to_['covaxin'],customized_score_with_similar_['covaxin'],sim_to_['b11529'],customized_score_with_similar_['b11529'],sim_to_['remdesivir'],customized_score_with_similar_['remdesivir'],sim_to_['imdevimab'],customized_score_with_similar_['imdevimab']
3760,3760,favipiravir,0.745560,1714,807,favipiravir,2009.0,2,['10.1016/j.ijid.2021.12.324' '10.1101/2022.02...,"[11, 10, 10]",...,0.330794,199.027577,0.393682,236.865319,0.244129,146.884156,0.886660,533.474289,0.527643,317.465410
1125,1125,ribavirin,0.915900,1215,644,ribavirin,1990.0,8,['10.3389/fpubh.2021.678785' '10.1007/s40121-0...,"[15, 10, 9]",...,0.343949,202.874602,0.342185,201.834407,0.262067,154.577553,0.858825,506.568987,0.529884,312.546860
3532,3532,arbidol,0.757475,476,201,arbidol,2008.0,3,['10.1097/md.0000000000024475' '10.12998/wjcc....,"[11, 10, 9]",...,0.354208,53.929058,0.365097,55.586938,0.235784,35.898652,0.787442,119.890035,0.580321,88.355348
3986,3986,hcq,0.732688,3834,828,hcq,2007.0,2,['10.1002/14651858.cd013587.pub2' '10.1159/000...,"[17, 16, 15]",...,0.293549,178.085950,0.398393,241.690956,0.190445,115.536637,0.757581,459.598276,0.478687,290.402985
2694,2694,lpv/r,0.808570,304,79,lpv/r,2006.0,4,['10.2139/ssrn.3878828' '10.1016/j.eclinm.2021...,"[10, 10, 9]",...,0.287413,18.359107,0.320018,20.441760,0.189206,12.085939,0.753080,48.104486,0.510181,32.588803
1347,1347,fvp,0.899167,79,19,fvp,2020.0,7,['10.1002/jmv.27724' '10.1007/s11357-021-00452...,"[9, 7, 7]",...,0.221699,3.787542,0.401177,6.853774,0.129638,2.214753,0.736792,12.587488,0.447479,7.644809
2628,2628,sofosbuvir/daclatasvir,0.812404,37,14,sofosbuvir/daclatasvir,2016.0,5,['10.1093/jac/dkab433' '10.1080/14787210.2021....,"[6, 3, 2]",...,0.277300,3.153918,0.355989,4.048892,0.189793,2.158639,0.701400,7.977484,0.537793,6.116675
4236,4236,cq,0.720511,1179,399,cq,2007.0,2,['10.1007/s00228-021-03133-w' '10.1016/j.bioph...,"[10, 10, 10]",...,0.258406,74.287637,0.299025,85.964832,0.190076,54.643799,0.691813,198.885018,0.375019,107.811796
693,693,baloxavir,0.947385,141,51,baloxavir,2018.0,9,['10.1186/s12879-021-06494-w' '10.3390/jof8010...,"[9, 6, 4]",...,0.424466,20.508787,0.311063,15.029521,0.309829,14.969913,0.688754,33.278247,0.508112,24.550276
2889,2889,repurposed,0.796839,1453,1215,repurposed,2010.0,4,['10.1177/20420986211041277' '10.1097/cce.0000...,"[6, 6, 5]",...,0.276272,267.475460,0.249281,241.343155,0.286585,277.460028,0.669983,648.650323,0.427422,413.812076


In [12]:
score_of_word_with_info_f.sort_values("sim_to_['omicron']",ascending=False)[:10]

,Unnamed: 0,word,score,cnt_in_all_articles,cnt_of_articles,index,first_year,quantile,doi,cnt_of_words,...,sim_to_['omicron'],customized_score_with_similar_['omicron'],sim_to_['covaxin'],customized_score_with_similar_['covaxin'],sim_to_['b11529'],customized_score_with_similar_['b11529'],sim_to_['remdesivir'],customized_score_with_similar_['remdesivir'],sim_to_['imdevimab'],customized_score_with_similar_['imdevimab']
536,536,delta,0.958818,6123,2873,delta,1996.0,9,['10.1101/2021.11.28.470293' '10.1016/j.jaut.2...,"[21, 18, 15]",...,0.907764,2500.603188,0.474822,1307.985865,0.840837,2316.239843,0.378127,1041.619685,0.579941,1597.554923
1974,1974,variant,0.855420,33030,13817,variant,1974.0,6,['10.1101/2021.06.27.21259602' '10.1001/jamain...,"[37, 22, 19]",...,0.873465,10323.784349,0.343954,4065.315314,0.871341,10298.677812,0.286001,3380.348145,0.450383,5323.236089
2103,2103,voc,0.845742,2130,1002,voc,2005.0,6,['10.1128/spectrum.01128-21' '10.21203/rs.3.rs...,"[14, 13, 12]",...,0.841291,712.938168,0.350858,297.328906,0.848639,719.165566,0.284262,240.892834,0.478249,405.284665
117,117,vocs,0.989776,1748,810,vocs,2004.0,10,['10.13227/j.hjkx.202106253' '10.1128/spectrum...,"[13, 12, 10]",...,0.747573,599.342624,0.317931,254.890738,0.749368,600.782229,0.238701,191.370834,0.435844,349.424390
1314,1314,sars-cov-2,0.901004,198867,75445,sars-cov-2,2019.0,7,['10.46234/ccdcw2021.215' '10.12114/j.issn.100...,"[32, 24, 22]",...,0.692939,47103.405805,0.332834,22624.793094,0.611325,41555.564721,0.399330,27144.995015,0.384575,26141.971965
2194,2194,vois,0.839535,96,62,vois,2012.0,5,['10.1101/2021.06.18.449083' '10.1128/spectrum...,"[4, 4, 3]",...,0.675445,35.157702,0.280292,14.589527,0.729397,37.965954,0.154564,8.045241,0.397791,20.705505
370,370,nabs,0.970900,1238,421,nabs,2001.0,10,['10.1101/2021.08.18.21262214' '10.1182/blood-...,"[19, 11, 10]",...,0.664283,271.525149,0.522992,213.772580,0.558347,228.223915,0.379249,155.017508,0.610662,249.607644
4991,4991,mutated,0.685041,1109,928,mutated,1988.0,1,['10.1016/j.compbiomed.2021.105170' '10.1016/j...,"[5, 5, 4]",...,0.646819,411.194548,0.201825,128.304106,0.663135,421.567226,0.243082,154.531924,0.364846,231.939430
3213,3213,neutralising,0.776231,939,501,neutralising,1977.0,3,['10.1016/s2666-5247(21)00025-2' '10.1016/s266...,"[17, 11, 7]",...,0.641128,249.329305,0.553493,215.249012,0.538285,209.334674,0.355734,138.342124,0.577982,224.772392
787,787,nab,0.940925,1345,447,nab,2004.0,9,['10.1097/01.ccm.0000806876.70744.05' '10.1016...,"[16, 15, 12]",...,0.637977,268.328987,0.539876,227.068435,0.527524,221.873243,0.341658,143.699135,0.569326,239.454763
